# SNIDdataset and SNIDsn Tutorial

In [1]:
import SNIDsn
import SNIDdataset
import numpy as np

## SNIDsn

SNIDsn is a Python class that loads the .lnw template files from SNID. An instance has the following fields:

header -- A dictionary containing information about the template.

continuum -- A numpy array containing the removed continuum.

phases -- A numpy array of the phases observed in the template.

phaseType -- 0 for phases measured relative to Vmax, 1 for date of discovery.

wavelengths -- A numpy array with the wavelengths for the observed spectra.

data -- A structured numpy array containing the spectra. Check dtype.names for column names.

type -- A string giving the type of the object in the template.

subtype -- A string giving the subtype (if any) of the object in the template.

In [20]:
sn04gt = SNIDsn.SNIDsn()

In [21]:
sn04gt.loadSNIDlnw('test/sn2004gt.lnw')

In [22]:
sn04gt.header

{'Nbins': 1024,
 'Nspec': 13,
 'SN': 'sn2004gt',
 'SplineKnots': 10,
 'SubTypeInt': 2,
 'TypeInt': 3,
 'TypeStr': 'Ic-norm',
 'WvlEnd': 10000.0,
 'WvlStart': 2500.0,
 'dm15': -9.99}

In [23]:
sn04gt.phases

array([ 16.4,  18.4,  19.4,  22.4,  43.3,  48.2,  70.1,  81.1, 100. ,
       131.8, 141.7, 167.6, 186.5])

Get column names for spectra structured array

In [24]:
print sn04gt.data.dtype.names
print sn04gt.getSNCols()

('Ph16.4', 'Ph18.4', 'Ph19.4', 'Ph22.4', 'Ph43.3', 'Ph48.2', 'Ph70.1', 'Ph81.1', 'Ph100.0', 'Ph131.8', 'Ph141.7', 'Ph167.6', 'Ph186.5')
('Ph16.4', 'Ph18.4', 'Ph19.4', 'Ph22.4', 'Ph43.3', 'Ph48.2', 'Ph70.1', 'Ph81.1', 'Ph100.0', 'Ph131.8', 'Ph141.7', 'Ph167.6', 'Ph186.5')


Use column names to get spectra at specific phases:

In [25]:
spec = sn04gt.data['Ph16.4']
print spec

[0. 0. 0. ... 0. 0. 0.]


Filter wavelengths to a certain range, and filter the spectra.

In [8]:
sn04gt.wavelengthFilter(wvlmin=4000, wvlmax=7000)
print sn04gt.wavelengths.shape
print sn04gt.data.shape

(414,)
(414,)


Replace 0.0 placeholder value used by SNID with np.nan

In [26]:
sn04gt.snidNAN()

Remove columns from the spectra matrix.

In [12]:
sn04gt.removeSpecCol('Ph16.4')

In [27]:
sn04gt.data['Ph19.4']

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [94]:
def findGaps(spec, wvl):
    nanind = np.argwhere(np.isnan(spec)).flatten()
    nanwvl = wvl[np.isnan(spec)]
    gaps = []
    gapStartInd = nanind[0]
    gapStartWvl = nanwvl[0]
    for i in range(0,len(nanind) - 0):
        ind = nanind[i]
        if ind == nanind[-1]:
            gap = (gapStartWvl, wvl[ind])
            gaps.append(gap)
            break
        nextInd = nanind[i + 1]
        if ind +1 != nextInd:
            gap = (gapStartWvl, wvl[ind])
            gaps.append(gap)
            gapStartInd = nextInd
            gapStartWvl = wvl[nextInd]
    return gaps

def largeGapsInRange(gaps, minwvl, maxwvl, maxgapsize):
    gapInRange = False
    for gap in gaps:
        gapStart = gap[0]
        gapEnd = gap[1]
        gapsize = gapEnd - gapStart
        if maxgapsize > gapsize:
            continue
        else:
            if gapStart < minwvl and gapEnd > maxwvl: gapInRange = True
            if gapStart > minwvl and gapStart < maxwvl: gapInRange = True
            if gapEnd > minwvl and gapEnd < maxwvl: gapInRange = True
    return gapInRange

def getInterpRange(wavelengths, minwvl, maxwvl, spec):
    wv = wavelengths[np.logical_and(wavelengths > minwvl, wavelengths < maxwvl)]
    wvStart = wv[0]
    wvEnd = wv[-1]
    wvFinite = wavelengths[np.isfinite(wavelengths)]

In [60]:
sp = np.array([np.nan, np.nan, 1, 1, 1, np.nan, 1, np.nan, 1, 1, np.nan, np.nan])
wv = np.arange(0,len(sp))+100

In [61]:
findGaps(sp, wv)

[(100, 101), (105, 105), (107, 107), (110, 111)]

In [62]:
print wv
print sp

[100 101 102 103 104 105 106 107 108 109 110 111]
[nan nan  1.  1.  1. nan  1. nan  1.  1. nan nan]


In [88]:
g = findGaps(sn04gt.data['Ph19.4'], sn04gt.wavelengths)

In [89]:
g

[(2501.69, 3689.57),
 (6031.16, 6031.16),
 (6298.19, 6298.19),
 (6766.7, 6766.7),
 (9543.68, 9543.68),
 (9569.56, 9993.24)]

In [93]:
largeGapsInRange(g, 4000, 7000, -1)

True

You can save your SNIDsn object using pickle (it automatically names the pickle file and puts it in the directory you specify.):

In [8]:
sn04gt.save('test/')

## SNIDdataset

For many applications, you will want to load many SNID templates and easily work with them in a dataset. This module facilitates your work by creating a dictionary of SNIDsn objects. 

Load the templates specified in snlist, located in the directory specified by pathdir.

In [9]:
SNIDset = SNIDdataset.loadDataset(pathdir='test/', snlist='snlist.txt')

In [10]:
SNIDset.keys()

['sn1997ef',
 'sn1998dt',
 'sn2008ax',
 'sn2004dk',
 'sn2011ei',
 'sn2007Y',
 'sn2011fu',
 'sn2004gt',
 'sn2010ay',
 'sn2004gq']

Get the type breakdown of your dataset:

In [11]:
typedict = SNIDdataset.datasetTypeDict(SNIDset)

In [12]:
typedict

{'IIb': array(['sn2008ax', 'sn2011ei', 'sn2011fu'],
       dtype='|S8'),
 'Ib': array(['sn1998dt', 'sn2004dk', 'sn2007Y', 'sn2004gq'],
       dtype='|S8'),
 'Ic': array(['sn2004gt'],
       dtype='|S8'),
 'IcBL': array(['sn1997ef', 'sn2010ay'],
       dtype='|S8')}

Use the type dictionary to create subsets of your dataset:

In [13]:
IIb_dataset = SNIDdataset.subset(SNIDset, typedict['IIb'])

In [14]:
IIb_dataset.keys()

['sn2011fu', 'sn2008ax', 'sn2011ei']